ref: https://python3.wannaphong.com/2018/12/named-entity-recognition-ner-pythainlp.html

# Instailation 

In [1]:
# !pip install nltk

In [2]:
!pip install transformers sentencepiece

In [3]:
# !pip install sklearn-pycrfsuite

In [4]:
!pip install pythainlp[full]

  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.1.6
    Uninstalling Keras-2.1.6:
      Successfully uninstalled Keras-2.1.6


In [5]:
# !pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite

In [6]:
# !pip install sklearn

In [7]:
# !pip install tensorflow

# Code

## import

In [8]:
import codecs
from tqdm import tqdm
# pythainlp
from pythainlp.tokenize import word_tokenize,subword_tokenize
#  sentence segmentator
from pythainlp.tokenize import sent_tokenize
from pythainlp.tag import pos_tag
from nltk.tokenize import RegexpTokenizer
import glob
import nltk
import re

import pandas as pd

# thai cut
thaicut="newmm"
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate,train_test_split
import pycrfsuite
from pythainlp.corpus.common import thai_stopwords

from transformers import (
    CamembertTokenizer,
    CamembertTokenizerFast, 
    AutoTokenizer,
    AutoModel,
    AutoModelForMaskedLM,
    AutoModelForSequenceClassification,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    pipeline,
)


2023-05-19 21:17:53.353701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-19 21:17:53.791253: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-19 21:17:53.791302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-19 21:17:53.791307: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [9]:
stopwords = list(thai_stopwords())
# stopwords

## Function 

### OG WangchanBERTa Function

In [11]:
#จัดการประโยคซ้ำ
data_not=[]
def Unique(p):
    text=re.sub("<[^>]*>","",p)
    text=re.sub("\[(.*?)\]","",text)
    text=re.sub("\[\/(.*?)\]","",text)
    if text not in data_not:
        data_not.append(text)
        return True
    else:
        return False

In [12]:
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[\[(.*?)\]\](.*?)\[\[\/(.*?)\]\]'
tokenizer = RegexpTokenizer(pattern) 
# ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')

In [13]:
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
    # text=text.strip().replace("FACILITY","LOCATION").replace("[AGO]","").replace("[/AGO]","").replace("[T]","").replace("[/T]","")
    # !! change pattern 
    text=re.sub("<[[^>]]*>","",text)
    text=re.sub("(\[\[\/(.*?)\]\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')# text.replace('>','>***') # ตัดการกับพวกไม่มี tag word
    text=re.sub("(\[\[\w+\]\])","***\\1",text)
    text2=[]
    for i in text.split('***'):
        if "[[" in i:
            text2.append(i)
        else:
            text2.append("[[word]]"+i+"[[/word]]")
    text="".join(text2)#re.sub("[word][/word]","","".join(text2))
    return text.replace("[[word]][[/word]]","")

In [14]:
# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=False):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text)
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    text=re.sub(' +', ' ',text) # remove multiple whitespace !new added
    tag=tokenizer.tokenize(text)
    # print("Text : ", text)
    # print("Tag : ", tag)
    j=0
    conll2002=""
    for tagopen,text,tagclose in tag:
        word_cut = word_tokenize(text,engine=thaicut) # ใช้ตัวตัดคำ newmm จาก var ด้านบน (PyThaiNLP)
        i=0
        txt5=""
        while i<len(word_cut):
            # print(i, "-- ", word_cut[i])
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
                # print("B-",tagopen)
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
                # print("I-",tagopen)
            else:
                txt5+=word_cut[i]
                txt5+='\t'+'O'
                # print('O')
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
        # print(txt5)
    if pos==False:
        return conll2002
    return postag(conll2002)

In [15]:
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine="perceptron", corpus="orchid")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text

In [16]:
# เขียนไฟล์ข้อมูล conll2002
def write_conll2002(file_name,data):
    """
    ใช้สำหรับเขียนไฟล์
    """
    with codecs.open(file_name, "w", "utf-8-sig") as temp:
        temp.write(data)
    return True
    
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
    """
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
    with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
        lines = f.read().splitlines()
    return [a for a in tqdm(lines)] # เอาไม่ซ้ำกัน if Unique(a)

In [17]:
def alldata(lists):
    text=""
    for data in lists:
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num = []
        try:
            sents_text = sentence_seg(data)
            sents_text = sentence_check(sents_text)
            # print("Split sents --",sents_text)
            # print("Len -- ", len(sents_text), "\nType -- ", type(sents_text))

            for sent in sents_text: 
                # pos = verb, noun (True or False)
#                 print("Sent -- ", sent)
                data_num=[]
                txt=text2conll2002(sent,pos=True).split('\n')
                for d in txt:
                    tt=d.split('\t')
                    if d!="":
                        if len(tt)==3: # pos=True
                            data_num.append((tt[0],tt[1],tt[2]))
                        else: # pos=False
                            data_num.append((tt[0],tt[1]))
                # print(data_num)
                # print("Append - ", data_num)
                data_all.append(data_num)
                # print("pass\n")

        except:
            print(data)
            print("not pass\n")
    # print(data_all)
    return data_all

def alldata_list_str(lists):
    string=""
    for data in lists:
        string1=""
        for j in data:
            string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
        string1+="\n"
        string+=string1
    return string

def get_data_tag(listd):
    list_all=[]
    c=[]
    for i in listd:
        if i !='':
            c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
        else:
            list_all.append(c)
            c=[]
    return list_all
  
def getall(lista):
    ll=[]
    for i in tqdm(lista):
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(i)
    return ll

### Added Function 

In [18]:
def write_to_xlsx(table, filepath):
    table.to_excel(filepath+'.xlsx')
    print("finished excel for ", filepath+'.xlsx')
    return True

In [19]:
def write_to_csv(table, filepath):
    table.to_csv(filepath+'.csv', encoding='utf-8-sig')
    print("finished csv for ", filepath+'.csv')
    return True

In [20]:
def write_to_txt (table, filepath):
    file = open(filepath+'.txt','w')
    for item in table:
        file.write(item+"\n")
    file.close()
    print("finished txt for ", filepath+'.txt')
    return True 

In [21]:
def list_to_txt(li, filepath):
    import pickle
    with open(filepath+'.data', 'wb') as f:
        pickle.dump(li, f)
    print("END")
#     return true

In [22]:
def sentence_seg(text, en="crfcut"):
  # option - thaisum
  return sent_tokenize(text , engine=en)

In [23]:
def check_label(lists):
    _tag = r'\[\[(.*?)\]\]'
    pas = True 
    new_text = ''
    checked_list = []

    for ch_tag in lists: # loop ของประโยคที่ตัดออกมาแล้ว 
        # print("-----------------") 
        # print(ch_tag)
        find_tag = re. findall(_tag, ch_tag)

        # ถ้่า tag ติดมาไม่ครบ
        if len(find_tag)%2 != 0:
            new_text += str(ch_tag)
            pas = False 
          # print("Concat -- ", ch_tag)
        # tag ครบ
        if (len(re. findall(_tag, new_text))%2 == 0) and (new_text!= '') and (pas!= True):
            pas = True
            # print("condition!!  ", pas)

        if pas == True: 
            if new_text != '': 
                checked_list.append(new_text)
                # print("Added new_text -- ", new_text)
                new_text = '' # re new_text to empty 
            else: 
                checked_list.append(ch_tag)
                # print("Added ch_tag -- ", ch_tag)

    return checked_list

In [24]:
def sentence_check(text):
    checked_list = [] 
    # print("Text : ", text, "\n--------")

    checked_label = check_label(text)
    # print(checked_list)

    # print("sentence_checked : ",checked_label , "\n")

    checked_list.append(list(checked_label))
    # print(len(checked_list)) # check จำนวนข่าวที่ loop

    return checked_label

In [25]:
def tag_check(lists):
    # get list from alldata_list
#     for text in lists:
    for t in lists: 
#         print(t[2])
        if '-' in t[2]:
            return True
    return False

## Data

In [26]:
dataset = '/home/crimex/CRIMEX/Dataset/last/final_data.txt'
dataset 

In [27]:
data1 = getall(get_data(dataset))

100%|███████████████████████████████████████| 3466/3466 [01:09<00:00, 49.53it/s]


In [28]:
print(len(data1))

3461


### split train / test / val

In [31]:
import numpy as np

# data is your entire dataset (including features and target variable)
data2 = np.array(data1)
# num_rows = data2.shape[0]

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Calculate the number of rows in the data
num_rows = data2.shape[0]

# Calculate the split points
train_split = int(num_rows * train_ratio)
val_split = int(num_rows * (train_ratio + val_ratio))

# Split the data
np.random.seed(50)
train_data, val_data, test_data = np.split(data2, [train_split, val_split])

# Print the lengths of the split data
print("NO of NEWS")
print("Train: ",len(train_data))
print("Val:   ",len(val_data))
print("Test:  ",len(test_data))

NO of NEWS
Train:  2768
Val:    346
Test:   347


### counting Token & Avg lenght

In [32]:
all_news = alldata_list(data2) #sent 

/tmp/ipykernel_8332/3324909410.py:5: FutureWarning: Possible nested set at position 2
  text=re.sub("<[[^>]]*>","",text)


In [33]:
len(all_news)

74383

In [118]:
tcri=0
tvic=0
tenf=0
tdate=0
tloc=0
titm=0
tact=0
twor=0
troot=0
ttrig=0

lcri=[]
lvic=[]
lenf=[]
ldate=[]
lloc=[]
litm=[]
lact=[]
lwor=[]
lroot=[]
ltrig=[]

for n in all_news:
    pretag ='start'
    for token in n:
#         print('pre//  ', pretag)
        # count lenght 
        try: 
            t = token[2].split('-')
            tag = t[1]
        except:
            tag='O'
    
        if tag!= 'O':
            if pretag != tag: #and pretag!= 'O':
#                 print('NEW LABEL - ', tag)
                if tag == 'Criminal':
                    tcri=1
                elif tag == 'Victim':
                    tvic=1
                elif tag == 'Enforcement':
                    tenf=1
                elif tag == 'Datetime':
                    tdate=1
                elif tag == 'Location':
                    tloc=1
                elif tag == 'Item':
                    titm=1
                elif tag == 'Action':
                    tact=1
                elif tag == 'worth':
                    twor=1
                elif tag == 'Rootcause':
                    troot=1
                elif tag == 'Trigger':
                    ttrig=1
            else:
#                 print('append - ', tag)
                if tag == 'Criminal':
                    tcri+=1
                elif tag == 'Victim':
                    tvic+=1
                elif tag == 'Enforcement':
                    tenf+=1
                elif tag == 'Datetime':
                    tdate+=1
                elif tag == 'Location':
                    tloc+=1
                elif tag == 'Item':
                    titm+=1
                elif tag == 'Action':
                    tact+=1
                elif tag == 'worth':
                    twor+=1
                elif tag == 'Rootcause':
                    troot+=1
                elif tag == 'Trigger':
                    ttrig+=1    
#             pretag = tag
                
        # last token in label        
#         if tag== 'O' and pretag!= 'O':
        if pretag!= tag and pretag!='O' and pretag!='start':
            if pretag == 'Criminal':
                lcri.append(tcri)
                tcri =0
            elif pretag == 'Victim':
                lvic.append(tvic)
                tvic=0
            elif pretag == 'Enforcement':
                lenf.append(tenf)
                tenf=0
            elif pretag == 'Datetime':
                ldate.append(tdate)
                tdate=0
            elif pretag == 'Location':
                lloc.append(tloc)
                tloc=0
            elif pretag == 'Item':
                litm.append(titm)
                titm=0
            elif pretag == 'Action':
                lact.append(tact)
                tact=0
            elif pretag == 'worth':
                lwor.append(twor)
                twor=0
            elif pretag == 'Rootcause':
                lroot.append(troot)
                troot=0
            elif pretag == 'Trigger':
                ltrig.append(ttrig)
                ttrig=0
            
#             print('Add - ', pretag,'-', tag, ' -- ', token)
#         print(tag)
        pretag = tag


In [125]:
def Average(lst):
    return sum(lst) / len(lst)


# find avg lenght and sum - token 
print('cri')
print(sum(lcri))
print(Average(lcri))
print('vic')
print(sum(lvic))
print(Average(lvic))
print('enf')
print(sum(lenf))
print(Average(lenf))
print('date')
print(sum(ldate))
print(Average(ldate))
print('loc')
print(sum(lloc))
print(Average(lloc))
print('item')
print(sum(litm))
print(Average(litm))
print('act')
print(sum(lact))
print(Average(lact))
print('worth')
print(sum(lwor))
print(Average(lwor))
print('root')
print(sum(lroot))
print(Average(lroot))
print('trig')
print(sum(ltrig))
print(Average(ltrig))

cri
37958
11.474607013301089
vic
27003
11.204564315352696
enf
31784
8.384067528356635
date
13634
6.7428288822947575
loc
32821
12.936933385888844
item
31329
10.37727724412057
act
26271
6.259471050750536
worth
17977
6.0753632984116255
root
4693
6.001278772378517
trig
7982
5.943410275502606


NO of NEWS
Train:  2311
Val:    289
Test:   289

In [35]:
train = alldata_list(train_data)
test  = alldata_list(test_data)
val   = alldata_list(val_data)

In [36]:
print("sent train: ", len(train))
print("sent test : ", len(test))
print("sent val  : ", len(val))

# sent train:  50521
# sent test :  8194
# sent val  :  8861

sent train:  55410
sent test :  9336
sent val  :  9637


### sentence

In [37]:
# write to txt file 
# format text tag
import re

def to_txt_bert(data, filepath):
    count =0
    uncount=0
    with open( filepath+'.txt', 'w') as file:
#         file.write("-DOCSTART- O\n\n")
        for item in data:

            for t in item: 
                if t[0] != ' ': # remove spacebar char
                    file.write(t[0])
                    file.write("|")
                    file.write(t[2])
                    file.write("\n")
            file.write('\n')

    file.close()
    
    print(filepath)
    print('OG len : ', len(data))

In [38]:
catt = 'final'
b_train_path = '/home/crimex/CRIMEX/Dataset/bert/sent/'+catt+'/bert_train'
b_test_path  = '/home/crimex/CRIMEX/Dataset/bert/sent/'+catt+'/bert_test'
b_val_path   = '/home/crimex/CRIMEX/Dataset/bert/sent/'+catt+'/bert_val'

to_txt_bert(train, b_train_path)
to_txt_bert(test, b_test_path)
to_txt_bert(val, b_val_path)

/home/crimex/CRIMEX/Dataset/bert/sent/final/bert_train
OG len :  55410
/home/crimex/CRIMEX/Dataset/bert/sent/final/bert_test
OG len :  9336
/home/crimex/CRIMEX/Dataset/bert/sent/final/bert_val
OG len :  9637


In [39]:
def li_to_txt_liform(data, filepath):
    count =0
    uncount=0
    with open(filepath+'.txt', "w") as f:
        for item in data:
            f.write("%s\n" % item)

    f.close()       
    print(filepath)
    print('OG len : ', len(data))

In [40]:
di = '/home/crimex/CRIMEX/Dataset/crf/sent/'
test_path = di + 'test_'+catt
train_path = di + 'train_'+catt

li_to_txt_liform(train, train_path)
li_to_txt_liform(test, test_path)

/home/crimex/CRIMEX/Dataset/crf/sent/train_final
OG len :  55410
/home/crimex/CRIMEX/Dataset/crf/sent/test_final
OG len :  9336


### Chunk

In [41]:
def chunk_news(data):
    conti = []
    news_chunk = []
    index = 0
    
    for item in data:
#         print("index", index)
#         print('ind len: ', len(item))
        
        # first element in data
        if index == 0:
            conti = item
#             print("First -- ", len(conti))
            
        else: 
            if len(conti) + len(item) > 200 or item[0][0] == '[::':
                news_chunk.append(list(conti))
#                 print('New ', len(conti), '+', len(item), '=', len(conti) + len(item), ' - ', item[0])
                conti = item
            else:
                conti += item
#                 print("---------add")

            #last element in data
            if index == len(data)-1 : 
                news_chunk.append(list(conti))
#                 print('Last')
        
        index+=1
                
    return news_chunk 


test_chunk = chunk_news(alldata_list(test_data))
val_chunk = chunk_news(alldata_list(val_data))
train_chunk = chunk_news(alldata_list(train_data))

print(len(train_chunk))
print(len(test_chunk))
print(len(val_chunk))

7577
1226
1247


In [42]:
di = '/home/crimex/CRIMEX/Dataset/crf/chunk/'
test_path = di + 'test_'+catt
train_path = di + 'train_'+catt

li_to_txt_liform(train_chunk, train_path)
li_to_txt_liform(test_chunk, test_path)

/home/crimex/CRIMEX/Dataset/crf/chunk/train_final
OG len :  7577
/home/crimex/CRIMEX/Dataset/crf/chunk/test_final
OG len :  1226


In [43]:
# data for BERT model
b_train_path = '/home/crimex/CRIMEX/Dataset/bert/chunk/'+catt+'/bert_train'
b_test_path  = '/home/crimex/CRIMEX/Dataset/bert/chunk/'+catt+'/bert_test'
b_val_path   = '/home/crimex/CRIMEX/Dataset/bert/chunk/'+catt+'/bert_val'

to_txt_bert(train_chunk, b_train_path)
to_txt_bert(test_chunk, b_test_path)
to_txt_bert(val_chunk, b_val_path)

/home/crimex/CRIMEX/Dataset/bert/chunk/final/bert_train
OG len :  7577
/home/crimex/CRIMEX/Dataset/bert/chunk/final/bert_test
OG len :  1226
/home/crimex/CRIMEX/Dataset/bert/chunk/final/bert_val
OG len :  1247
